In [1]:
import numpy as np
import torch
import torch.nn as nn
from torch.autograd import Variable

import rnn
import train
import brainfuck

In [2]:
embedding_size = 10
hidden_size = 35
output_size = len(rnn.char.keys())
n_layers = 2
batch_size = 64
GAMMA = 0.99 # for exponential moving avarage constant from paper




In [3]:
print(rnn.token_num)

9


In [4]:
model = rnn.BFgen(rnn.token_num, embedding_size, hidden_size, output_size, n_layers, batch_size)

In [5]:
print(train.objective_PG(model, lambda x : np.array([1] * model.batch_size)), model.entropy)

(Variable containing:
-13127.1816
[torch.FloatTensor of size 1]
, Variable containing:
 13258.7939
[torch.FloatTensor of size 1]
)


In [6]:
print(train.objective_PQT(model))

Variable containing:
-206.8125
[torch.FloatTensor of size 1]



In [7]:
print(model.pqt_programs)

['-[]]><,+<,--[+..+>.+]<+,<<]-.]-><-<>-..][+]+.>+-[]-,>-+>[>],][+]][,-[+<+-+>.,,].,,+<]<<<>]-,<,][+,<,'
 '.,].,],]<[+]]+>]<,,,>.>-,.-][,<,+>,[,<<-[-.,.[+-,]][<>-,<...],-+,+]><.,<,+]<..<,-[+<-.,<-<-.>-]+[-]>'
 '[.++-<-<]-..[+<]+.[<-<],<[].<<.-]+<<]<[+]<>+[]++--.<[..-],[<[>><[]<<[>>>[->,+<+[++.-<<,>>.-<]][,-,..'
 '.]+.<.]><><++[>,.].+..[.,.<]+,<]>,,>[[.<.[>>++[]<,,,<--],<><,<[+][-,<]<>-+>.<-<<,[>-,[[,,[,>+,+,,,]['
 '<-,>+,>+-,,++--<+.<+[>,]],.+]-<]-+><>-+,<<.->,[-.[,[],,<+]>,[.>--++[[]>[<,><<->-],+<.-[<>[>-[<<[->[-'
 '[]-]-.>[,<+>><+>.-<<<<].-<]]-<<],+<-][<-<->]<+>,++<>.-<.-].][,.+[[],++].[[.>,<<+]-]]><>[><>--.[<,,+['
 '><.--[-,,[+-+[[,.,[.,+->[-]-,-]<,><..<<.<-<,[][<-]<<-<+>.><+.>]<<>-.+->[]>]+++<<<>,,]+->[-.+[+.+++<.'
 '],]],<]+<+-,++,<--]>[>,.[>]]>]<],>-+><><[>[>-<+[+<>++]-<<-,.>>.[<,[,-][,->.+><,<[-,<>,+.,[<<]],>+>.<'
 '<+.<[>,[<,[,],+[+<,++,,+]]><],+>+[[]].],.,>-,,><<.,<]-[,[>.[>--<]<.--<-,,<,<][,,<+,<[,-[,[.>><[]<],['
 '>-,,+,+[->.,[,<[]<][+.+[<,.<,.,[>,][>,>--]<,+[<[+,<,.<+,,-,,->

In [8]:
bfAdd = ",>++++++[<-------->-],[<+>-]<++++++++."
bfAdd2 = ",>++++++[<-------->-],[<+>-]<."
bfAdd3 = ",>++++++[<-------->-],[<+>-]<++..."
bfAdd4 = ",>++++++[<-------->-],[<+>-]<+++............"
addBatch = [bfAdd, bfAdd2, bfAdd3, bfAdd4]
bfAddIn = ['11', '12', '23']
bfAddOut = ['2','3','5']

In [9]:
reward = train.batch_reward(bfAddIn, bfAddOut)

In [10]:
reward(addBatch)

array([  74.4,   76.8,  -77.4, -768.9])

In [11]:
def reverse(s):
    return s[::-1]

reverse_in, reverse_out, scaling_factor = train.gen_tests(reverse)
print(reverse_in, reverse_out, scaling_factor)

(['\xae\x81C\x90\x0f\xe0\x13E\xc7\x11xb\xd2T', 'd\x07\x1e\x05', 'sV\xef\x85\xe53KD \xc4', '\x1d\xc31F\xc2-\\\x8f\xed\xefU}3\xc6', '\x8f\x1e\xd4,B9BL\xf9\xdd\x9a\xd7\xb4\xd5\xc6', '\n\xd9\x98\xcdV\x84\x14\x867w', '\xef\xd6', "\xbb\xe3\x93'iH\xe3Y\x86", 'S-\xfei>f\x94\x8d\xc3', '\x14\x1c4', '\x8dQ\xb5\x99', '\x0c\xe2&Y\xd4\x84\x9d', '\x05-', 'na\xaa\x8d\x81v\xef9j\x12\x1bi\xfe\xa2\xbf', 'g\x88W\x0c\xda\xb4v\xd8?\x16h\xa5\xe6', ";\xf0v\xdb'`4\x85X;\x1c7[\xe1\xd1", '\xda\xf6\x1a\x94\xf1*', '\xe2\x04\xc7\xbf\x82y|o\xd4\xd8', '\xd1\xcb\xa0\xa1\xe4\xd2\xdc', ':E\xcf\xc0\xc7\x9e', '\xdf\xf6<\x0b\xca', '<\x9f\x8dp\x8a\x13t\x94\xce', '`\x0ef2\xc6Z', '$\xa3_\xf8G\xd88b\x82\xfa\n\t\xe2\xb9', 'Zd4$\xaa\xcd\x0e5\xff\xdc', '"\x9d%A\xf6u\xbc\xfdO\x87\xac`', '\xb7+', '\xa2H\x8fE\xda}', '\xa24;\x9d\xa8\xa4\xd7\x93\xbcg', '\x1b\xfb\x9b]\x10\x02X\x14\x1e&\n\x02', 'K\xf3\xe2i)\x88\x88YU\xe0', '^\xd5\x8a\xd6\x15t\xce\xb9}', '\xaf\xde?\xfa,', '\x95', '\x03\xa4Z\xc6', "\xbb\x07'?\xcb@\xb4\x85\x99\xdc_\xd0", '

In [14]:
reverse_reward = train.batch_reward(reverse_in, reverse_out, scaling_factor=scaling_factor)

In [15]:
model.clear_pq()
model.init_weights()
train.train_pqt_pg(model, reverse_reward, NPE=20000000)

Epoch Obj  Sample
----- ----- ------
Variable containing:
 46076.3789
[torch.FloatTensor of size 1]

Variable containing:
-203.6339
[torch.FloatTensor of size 1]

Variable containing:
 203.8103
[torch.FloatTensor of size 1]

-0.025633749066755755
1.167681
pqt [('[[<+[[<[<++<+-.<[,,[-.-.>-+,],[,-->><-+[-[[.,<.[-<[>[.[>,[-<--+],<--,---++[.<[.,[[>,[+[.]]+],-[[-<>.', -0.8515237221977883), ('>>[.-].].>]]>]..-,]>[,][..+>,[,+.-][[<-,.][][>]+--,,>--+-..<]-[<.,,.,.>-.[.+.[-[.,+.,<,..,]>][.,-.,]', 0.0), ('..+]->[[-,..>-+..-]].-[[[<[-+>+>.[-[<-....[,,>[+>[<>+].,<],-[.,+.[[.-..-[[,++-,<.[.>.+-[[+[+,,[,.>..', 0.0), ('-.[,,],]].<<[][[.][.[.+<]][-->,][[>+]-[[+[<.]...[-[>.[+[.<.<><-,,]].<+...-.,.+.,.,[<<,-[,,]+-++[[+,.', 0.06420131471475245), ('[-,--..<<-.+++,>[-+..[[[,[<>>>,,,[+->>[]<],[.>.+>[[][-+-.<<]+>-.[>>+,[[[+>...<,<[->.]<-,.--[><[+.[+>', 0.22043266210103038), ('][.[<>,-[,,.+.]+]]>+.,][,.-..[,-<-+.-,[-.<[.-[+,[,][<-.].<[,.,-[+[>+>+-->.+,]+]<.[-.][]>>.--],,[+-,]', 0.2611159760618246), (',.,.[>]+

Variable containing:
-1350.1426
[torch.FloatTensor of size 1]

Variable containing:
-137.0004
[torch.FloatTensor of size 1]

Variable containing:
 145.3448
[torch.FloatTensor of size 1]

-1.4814671936701573
0.931415
pqt [('---,[<<<.>>,],]>>,>],,.>>>,,,.....,.,.,.[.,.-,,,...,,]..]..][.,,.,,,.,,]..[,,,.[.[,.[,+,.,.,,,-.,,,]', 0.7103764490135712), ('---,[<<<.>,],,[]]]]->.,.].-,...,.,,..]....-..,,,],.>,.,[>..-.,,,[>]....[,.+.,,,>,,.....+,..>[..-,,],', 0.7103764490135712), ('---,[[[<<.<>>,]]...-[,]]]]]..,.....[,>].,...,>,,[...,-..]..-.,.,.>.,,,...,.]..[>].,.][..,,..],..[...', 0.7103764490135712), ('---,[[<<.>,]>>,>>]>>>],>.,.].,.....,>.>,...,.,.,>.[,,,..,[,..,,.[,,..,,,,.,,,.,,..,]..[...-,.[.],.,,', 0.7103764490135712), ('--,[[[<<.>,],.,,],]]>]]].,,..]......]..[,.[,,,.,,,..,.[....,>.,+.,[..,].[..,...[[,..[,....,,,,.]..,,', 0.7103764490135712), ('--<[,[<.<<<>,]]>]<,..,].,]],].,,.>.>>.>.[.>,.>.]>]...],[.,-][[.[,,,.]+,,[...,,.]...,>,,,]->..,].,,.,', 0.7103764490135712), ('---,<[[<.>,],<,]]]].>

Variable containing:
 304.1094
[torch.FloatTensor of size 1]

Variable containing:
-121.6573
[torch.FloatTensor of size 1]

Variable containing:
 124.4300
[torch.FloatTensor of size 1]

-0.7023235591932772
0.930906000001
pqt [('-,[-[[[<<<<.>,]..],],.].,,],..,]]..>,].,.].]>,.>]]>...>.,.,.,..>>..].,.,.,],][.,.,]+,],.,,..,..[,.,+', 0.7118490237811007), ('-,[[-<[<<<.>,],.]],].]]>,],..],>],.>-,].,]]..,>,]..]]..,..,>+..,..,,].>,.....].]>..].,[,.,.,,,.,]],,', 0.7118490237811007), ('-,[[-[<[<<<.>>,]>,,],.]],]]],]..].],,].]...,],.]>>..]]]>,,.>.],],]],.],.,>.,,]..]..,,,,,.,..,,,.,[..', 0.7118490237811007), ('----,[[-[<<.<<>>,],.],].,,,]],]>,].]],.,.,]],]>,,].,+>...,].[..>],..,,.][,.,.>,.>>]]..],],.[,,>,],,,', 0.7118490237811007), ('---,[[-[<<<<.>,],.],.,],]]],>..],,,..,,,]],.].,.].>,.]......,]]]].]]].,,,.,.],,.,],,.[,...,]].[,,,,.', 0.7118490237811007), ('---,[[-<[<<<.>,],.]],]].,,]],,],]]>]]>,,,],,.[].>],.]].,.,.>.],,......,+>.,.]].>..,,].,..,,,.,,,[,>,', 0.7118490237811007), ('-,[-<[<<<<.>,],.

Variable containing:
 921.2056
[torch.FloatTensor of size 1]

Variable containing:
-105.3259
[torch.FloatTensor of size 1]

Variable containing:
 107.0675
[torch.FloatTensor of size 1]

-0.4315127775139442
0.943962000001
pqt [('---,[[-[[[<<<<.>>,]]],.],.]]],.].,.].]],.]>>,],.],[.>>,].,.>,.>,,..,.,.,.],..>,>,.>>,],],.,.].]],[[,', 0.7118490237811007), ('---,[[-[[<<<.>>>,]],.]],.]..,.]],.]]>,.],.]],...,..,.,],.,..,.].,>.,>..,.>]].[.,>.,,.,.]>[>,.,.]]>,>', 0.7118490237811007), ('---,[[-[[<<<<.>>,]],.]].,].,]]]],.]],],.,]]],..,>.,..[,..].>,.,.,.,.>.[,..,>.,].],..,.>],>,[],.,.,,.', 0.7118490237811007), ('-[,[[[-[<<<.>>>,],.]]],]]],.,].]],.]]>.]],],.,]].,.,.],.,.,.],.[]>,>>.,]>>],]>,.],.[,..,]],.],.,.[,,', 0.7118490237811007), ('----,[[[-[[<<<.>>>,]]],.].],]]],.,.,...]]]>,.,..],>],,....,.>>>,][].,>.,[>,],.>,[]>[,[,.,],..,>>,,]]', 0.7118490237811007), ('---[[[-,[-[[<<<<<.>>>,]],.].].,.]][,.]],..],.]]].,],.,.]],>,].]]>,>>,>>>,.,]].,,.],.>],][,.],,.>.,.,', 0.7118490237811007), ('--,[[[-[<<<<.>,]

Variable containing:
 1942.7689
[torch.FloatTensor of size 1]

Variable containing:
-72.0806
[torch.FloatTensor of size 1]

Variable containing:
 74.8266
[torch.FloatTensor of size 1]

-0.3693992211843367
0.997081000001
pqt [('---[[,[[[[<[-+[[-[[[++[[+[[<[[[+[[<[+[--<[<[[--<<.>,]],]],.]],]],.],.],]]]],.],.],]],.],.]].,.],]],]', 0.7146130466197537), ('---[[[,[[-+[[[-[+[[+[+[+[--+[[+[---<[<[<.>,]]],],.]],,.]],.],.],.,]]],.],.],.],.],]]],.],]]],]],]],.', 0.7146209003518471), ('--[,[[[[[-[--+[[++[[[[[+[+[[+[+[[+[+<[--[---<<<.>,]],]],..],.],.,.],]],.],,]],.,]],.],.]]],.]],.]],.', 0.7146621324453379), ('--[,[[--[+[-[+[[[-++[[+[[[+[++[----[<[<<.<>>,]]],]],.],]],]],.],>],.]],.]],.],.],.]],.,.],.],.>,.],.', 0.7146709678939431), ('---,[[-[+----[-[+++[+[+[+[[--+<<<.>,],],.]],.,..]],..,]]],.],.],.,.]],.]],.],.]],.],.,.],.]],..],]]]', 0.714746560065343), ('----[[[,[[[[-[++[[+[[++[+[-[<[-[---<[<<[<.>,],]],]],.],.],]],.]]],.],.],.],.,.]],.]]],.],]],.],..,.]', 0.714777974993717), ('--[[,[[[+[-[-[[+[-[

...

Variable containing:
-9.8801
[torch.FloatTensor of size 1]

Variable containing:
-6.9376
[torch.FloatTensor of size 1]

Variable containing:
 6.9044
[torch.FloatTensor of size 1]

0.6959897719738568
0.962803000002
pqt [('-----[[,[[[[[-[[[+[+[+[+[+[+[+[+[+[+[+[----------<<<<.>,]]]]],.],.,.]],.,.,.]]],.]],.]],.]],.]],.]]]', 0.7185585652802211), ('----[-,[[[[[-[[[+[+[+[+[+[+[+[+[+[+[+[----------<<<<.>,]]]]],.],.,.]],.,.,.]]]],.],.]],.]],.]],.]]],', 0.7185585652802211), ('-----[,[[[[[-[[[+[+[+[+[+[+[+[+[+[+[+[----------<<<<.>,]]]]],.],.,.]],.,.,.]]],.]],.],.]]],.]],.]]],', 0.7185585652802211), ('-----[,[[[[[-[[[+[+[+[+[+[+[+[+[+[+[+[----------<<<<.>,]]]]],.],.,.]],.,.,.]]],.]],.]],.]],.]],.]],.', 0.7185585652802211), ('-----,[[[[[[-[[+[+[+[+[+[+[+[+[+[+[+[----------<<<<.>,]]]]],.],.,.]],.,.,.]]],.]],.]],.]],.]]],.]],.', 0.7185585652802211), ('-----[,[[[[[[-[[+[+[+[+[+[+[+[+[+[+[+[----------<<<<.>,]]]]],.],.,.]],.,.,.]]],.]],.]],.]],.]],.]],.', 0.7185585652802211), ('----[-[[,[[[[[-[+[+[+[+

Variable containing:
 42.6332
[torch.FloatTensor of size 1]

Variable containing:
-4.9813
[torch.FloatTensor of size 1]

Variable containing:
 6.7978
[torch.FloatTensor of size 1]

0.7013327193416392
0.966201000003
pqt [('--------[[,[[[[[[[[[[-[[[[+[+[+[+[+[+[+[+[+[+[+[----------<<<<.>,]]]]],.],.,.]],.,.,.]]]]],.]]],.]]]', 0.7185585652802211), ('-------[[,[[[[[[[[[[-[[[[[+[+[+[+[+[+[+[+[+[+[+[----------<<<<.>,]]]]],.],.,.]],.,.,.]]]]],.]]],.]]]', 0.7185585652802211), ('-------[-[[,[[[[[[[[[-[[[[+[+[+[+[+[+[+[+[+[+[+[----------<<<<.>,]]]]],.],.,.]],.,.,.]]]]],.]]],.]]]', 0.7185585652802211), ('--------[[[,[[[[[[[[[-[[[+[+[+[+[+[+[+[+[+[+[+[----------<<<<.>,]]]]],.],.,.]],.,.,.]]]]],.]]],.]]],', 0.7185585652802211), ('------[-[[[[,[[[[[[[[[-[[[[+[+[+[+[+[+[+[+[+[+[+[----------<<<<.>,]]]]],.],.,.]],.,.,.]]]],.]]],.]]]', 0.7185585652802211), ('--------[[[,[[[[[[[[-[[[[[+[+[+[+[+[+[+[+[+[+[+[----------<<<<.>,]]]]],.],.,.]],.,.,.]]]]],.]]],.]]]', 0.7185585652802211), ('-------[[[[,[[[[[[[[[[

Variable containing:
-1.6276
[torch.FloatTensor of size 1]

Variable containing:
-3.2789
[torch.FloatTensor of size 1]

Variable containing:
 4.4358
[torch.FloatTensor of size 1]

0.7111659363523212
0.961928000004
pqt [('-----[[,[[[[[[[[[-[+[+[+[+[+[+[+[+[+[+[+[----------<<<<.>,]]]]],.],.,.]],.,.,.]]],.]],.]],.]],.]],.]', 0.7185585652802211), ('-----[[[,[[[[[[-[[[+[+[+[+[+[+[+[+[+[+[+[----------<<<<.>,]]]]],.],.,.]],.,.,.]]],.]],.]],.]],.]],.]', 0.7185585652802211), ('-----[[,[[[[[[[-[[[+[+[+[+[+[+[+[+[+[+[+[----------<<<<.>,]]]]],.],.,.]],.,.,.]]],.]],.]],.]],.]],.]', 0.7185585652802211), ('-----[[,[[[[[[[-[[[+[+[+[+[+[+[+[+[+[+[+[----------<<<<.>,]]]]],.],.,.]],.,.,.]]],.]],.]],.]],.]],.]', 0.7185585652802211), ('-----[[[,[[[[[[[-[[+[+[+[+[+[+[+[+[+[+[+[----------<<<<.>,]]]]],.],.,.]],.,.,.]]],.]],.]],.]],.]],.]', 0.7185585652802211), ('-----[[,[[[[[[[[[-[+[+[+[+[+[+[+[+[+[+[+[----------<<<<.>,]]]]],.],.,.]],.,.,.]]],.]],.]],.]],.]],.]', 0.7185585652802211), ('-----[[[,[[[[[[-[[[+[+[

Variable containing:
-1.8105
[torch.FloatTensor of size 1]

Variable containing:
-4.4617
[torch.FloatTensor of size 1]

Variable containing:
 3.8810
[torch.FloatTensor of size 1]

0.7111122458905664
1.018161
pqt [('-------,[[[[[[-[[+[+[+[+[+[+[+[+[+[+[+[----------<<<<.>,]]]]],.],.,.]],.,.,.]]],.]],.]],.]],.]],.]],', 0.7185585652802211), ('------[,[[[[[[-[[+[+[+[+[+[+[+[+[+[+[+[----------<<<<.>,]]]]],.],.,.]],.,.,.]]],.]],.]],.]],.]],.]],', 0.7185585652802211), ('------[,[[[[[[-[[+[+[+[+[+[+[+[+[+[+[+[----------<<<<.>,]]]]],.],.,.]],.,.,.]]],.]],.]],.]],.]],.]],', 0.7185585652802211), ('-------[,[[[[[-[[+[+[+[+[+[+[+[+[+[+[+[----------<<<<.>,]]]]],.],.,.]],.,.,.]]],.]],.]],.]],.]],.]],', 0.7185585652802211), ('----[--,[[[[[-[[[+[+[+[+[+[+[+[+[+[+[+[----------<<<<.>,]]]]],.],.,.]],.,.,.]]],.]],.]],.]],.]],.]],', 0.7185585652802211), ('-------,[[[[[-[[+[+[+[+[+[+[+[+[+[+[+[----------<<<<.>,]]]]],.],.,.]],.,.,.]]],.]],.]],.]],.]],.]],.', 0.7185585652802211), ('------[,[[[[[[-[[+[+[+[+[+[+[

KeyboardInterrupt: 

In [21]:
import bfCompiler
brainfuck.BF("----[,[[[[[-[+[+[+[+[+[+[+[+[+[+[+[----------<<<<.>,]]]]],.],.,.]],.,.,.]]],.]],.]],.]],.]],.]],.]],", "aasdsasdef")

'aaaaaaaaaa'

In [ ]:
expected_code = "><.><"
program_batch_code = [".....", "..<..", "[][][][][[][[][][][][][][]]]", "]"]

In [ ]:
simple_rew = train.simplified_batch_reward(expected_code)

In [ ]:
simple_rew(program_batch_code)

In [ ]:
training_expected_code = ">>>.." * 20
print len(training_expected_code),training_expected_code

In [ ]:
training_rew = train.simplified_batch_reward(training_expected_code)
training_rew(program_batch_code)

In [ ]:
def ultra_simple_rew(expected_code):
    def hamming_distance(seq1, seq2):
        assert len(seq1) == len(seq2)
        return sum(elem1 != elem2 for elem1, elem2 in zip(seq1, seq2))
    
    code_len = float(len(expected_code))
    def reward_code(generated_code_batch):
        rewards = np.array(map(lambda x: 1 - 2*(hamming_distance(x, expected_code) / code_len), generated_code_batch))
        return rewards
    return reward_code

from itertools import takewhile
def prefix_rew(expected_code):
    def common_prefix_len(seq1, seq2):
        return len(list(takewhile(lambda (a, b): a == b, zip(seq1, seq2))))
    
    code_len = float(len(expected_code))
    def reward_code(generated_code_batch):
        rewards = np.array(map(lambda x: 2*(common_prefix_len(x, expected_code) / code_len) - 1, generated_code_batch))
        return rewards
    return reward_code

In [ ]:
simple_rew = prefix_rew(training_expected_code)
sample1 = ">" * 100
sample2 = "." * 100
sample3 = "[]" * 50
sample4 = "[]><+-.,[]"  * 10
samples = [sample1, sample2, sample3, sample4]

In [ ]:
simple_rew(samples)

In [ ]:
model.clear_pq()
model.init_weights()
train.train_pqt_pg(model, simple_rew, exp_code=training_expected_code, NPE=10000000)

In [ ]:
print(model.get_probs(training_expected_code))

In [ ]:
print(model.sample())